In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GOOG_test.csv")

In [4]:
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GOOG_test.csv")

In [5]:
test.describe()

,Open,High,Low,Close,Adj Close,Volume
count,253.000000,253.000000,253.000000,253.000000,253.000000,2.530000e+02
mean,2702.674255,2730.161156,2672.508566,2700.858899,2700.858899,1.303822e+06
std,207.658815,202.614255,208.531935,205.434607,205.434607,5.369361e+05
min,2102.840088,2127.899902,2044.160034,2116.790039,2116.790039,5.933000e+05
25%,2596.669922,2618.074951,2550.070068,2584.800049,2584.800049,9.462000e+05
50%,2742.194092,2767.989990,2714.000000,2740.090088,2740.090088,1.182100e+06
75%,2864.020020,2885.959961,2834.699951,2856.060059,2856.060059,1.512400e+06
max,3037.270020,3042.000000,2997.750000,3014.179932,3014.179932,4.487500e+06


In [6]:
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
248,2022-06-03,2319.850098,2327.290039,2273.360107,2291.280029,2291.280029,1252600
249,2022-06-06,2334.850098,2387.969971,2330.565918,2340.209961,2340.209961,1189300
250,2022-06-07,2312.959961,2354.972900,2302.510010,2344.590088,2344.590088,1320700
251,2022-06-08,2337.530029,2372.919922,2333.935059,2344.760010,2344.760010,1127200
252,2022-06-09,2326.830078,2367.000000,2297.340088,2298.360107,2298.360107,1157100


In [7]:
df = list(df["Close"])
test = list(test["Close"])

In [8]:
df.extend(test)
len(df)

506

Stationarity\ Subtract the previous value from the current value. Now if we just difference once, we might not get a stationary series so we might need to do that multiple times. And the minimum number of differencing operations needed to make the series stationary needs to be imputed into our ARIMA model.

ADF test\ We'll use the Augumented Dickey Fuller (ADF) test to check if the price series is stationary. The null hypothesis of the ADF test is that the time series is non-stationary. So, if the p-value of the test is less than the significance level (0.05) then we can reject the null hypothesis and infer that the time series is indeed stationary. So, in our case, if the p-value > 0.05 we'll need to find the order of differencing.

In [9]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(df)
print("ADF Statistics:", result[0])
print("p Value", result[1])

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


ADF Statistics: -2.3826710720059383
p Value 0.14668722059719014


In [10]:
# Now to make data Stationary we subtract previous value from current till we get p < 0.05 
# To find out how many times we need to repeat the above step we use pmdarima
!pip install pmdarima

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 5.4 MB/s 
     |████████████████████████████████| 9.8 MB 31.9 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [ ]:
from pmdarima.arima.utils import ndiffs

In [ ]:
d_value = ndiffs(df,test = "adf")
print("d value:", d_value)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
from pmdarima import auto_arima

In [ ]:
x_train= df[:-253]
x_test = df[-253:]
print(len(x_train),len(x_test))

In [ ]:
step_fit = auto_arima(df,trace=True,suppress_warnings=True)

In [ ]:
import statsmodels.api as sm

In [ ]:
model = sm.tsa.arima.ARIMA(df, order=(1,1,1))

In [ ]:
result = model.fit()
print(step_fit.summary())

In [ ]:
from statsmodels.graphics.tsaplots import plot_predict

In [ ]:
plot_predict(result, start = 700, end = 1270, dynamic = False);

In [ ]:
#Now lets Forecast
start = len(x_train)
end= len(x_train) + len(x_test) - 1
fc = result.predict(start=start,end=end)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt
sqrt(mean_squared_error(x_test,fc))

In [ ]:
r2_score(x_test,fc)

In [ ]:
#Plotting 
plt.figure(figsize=(10,6), dpi=100)
plt.style.use("dark_background")

#plt.plot(x_train, label='training', color = "green")
plt.plot(x_test,label='actual', color = "red")
plt.plot(fc , label='forecast', color = "blue")
plt.title('Forecast vs Actuals')

plt.legend(loc='upper left', fontsize=8)
plt.grid()
plt.show()